In [1]:
from experiment import Experiment
from utils.behavior_data import BehaviorData
from visuals import Plotter
import torch
import numpy as np
from utils.state_data import StateData
import matplotlib.pyplot as plt

In [ ]:
model, learning_rate = "AdaptableLSTM", .025
# model, learning_rate = "BasicNN", .0024
# model, learning_rate = "LogisticRegressor", .003
epochs = 1000
seed = 1
include_state = True
estate = True
fullq = True
respond_perc = .50
fullseq = False
insertpreds = True
noise = .05
smooth = 0

    
np.random.seed(seed)
torch.manual_seed(seed)


e = Experiment(
    numValFolds = 5,
    epochsToUpdateLabelMods = 5,
    data_kw={"minw": 2,
            "maxw": 29,
            "include_state": include_state,
            "include_pid": False,
            "expanded_states": estate,
            "top_respond_perc": respond_perc,
             "full_questionnaire": fullq,
             "full_sequence": fullseq,
             "insert_predictions": insertpreds,
             "one_hot_response_features": True
            },
    model=model,
    model_kw={
        "lossfn": "MSELoss",
        # "lossfn": "NDCG",
        # "lossfn": "CrossEntropyLoss",
        "hidden_size": 200,
        "lr_step_mult": .9, 
        "lr_step_epochs": 100,
        "opt_kw": {
            "lr": learning_rate,
        },
        "labelSmoothPerc": smooth,
        "gaussianNoiseStd": noise
        
    },
    train_kw={
        "epochs": epochs,
        "n_subj": 500,
        "rec_every": 5,
    })


print(len(e.bd.test))
print(len(e.bd.train))

report = e.run()






54
214
0	 train loss: 0.1994 train acc: 51.598% test acc: 51.149%
5	 train loss: 0.2011 train acc: 51.598% test acc: 51.149%
10	 train loss: 0.1875 train acc: 51.598% test acc: 51.149%
15	 train loss: 0.1851 train acc: 51.598% test acc: 51.149%
20	 train loss: 0.1842 train acc: 51.598% test acc: 51.149%
25	 train loss: 0.1832 train acc: 51.598% test acc: 51.149%
30	 train loss: 0.1818 train acc: 51.598% test acc: 51.149%


In [ ]:

print (np.mean(report['train_metrics'], axis=0))
labels = report["metric_labels"]
print(report['train_metrics'][-1, labels.index("Acc")])
print(report['test_metrics'][-1, labels.index("Acc")])

splot = plt.plot(report["rec_epochs"], report["train_metrics"][:, labels.index("Acc")], label="Train Acc.")
splot = plt.plot(report["rec_epochs"], report["test_metrics"][:, labels.index("Acc")], label="Test Acc.")
splot = plt.plot(report["rec_epochs"], report["train_metrics"][:, labels.index("MSE")], label="Train MSE")
splot = plt.plot(report["rec_epochs"], report["test_metrics"][:, labels.index("MSE")], label="Test MSE")
plt.title("Train/Test Performance Over Training")
plt.legend()
plt.ylabel("Metric")
plt.xlabel("Training Epoch")
plt.savefig("simpleNotebookAccPlot.png")

plt.clf()